In [352]:
import requests
from lxml import html

In [350]:
sothebys_auctions_to_scrape = []
all_sotheby_auctions = []

In [437]:
artpredict_path = "/Users/denizthemenace/dev/artcollect/"
datapath = artpredict_path + "data/"
auction_list_html = datapath + '10.htm'

In [438]:
with open(auction_list_html, "r") as f:
    page = f.read()
html_tree = html.fromstring(page)

In [439]:
sothebys_auction_urls = html_tree.xpath('//a/@href['
                                           'contains(.,"www.sothebys.com/en/auctions/")'
                                           'and contains(.,".html") '
                                           'and not(contains(., "results.html"))]')
sothebys_auctions_to_scrape = list(set(sothebys_auction_urls))

In [440]:
sothebys_auctions_to_scrape

['http://www.sothebys.com/en/auctions/2017/prints-multiples-l17161.html',
 'http://www.sothebys.com/en/auctions/2017/fine-jewels-l17058.html',
 'http://www.sothebys.com/en/auctions/2017/irish-art-l17134.html',
 'http://www.sothebys.com/en/auctions/2017/vivien-leigh-collection-l17148.html',
 'http://www.sothebys.com/en/auctions/2017/watches-sale-l17054.html',
 'http://www.sothebys.com/en/auctions/watch-live.html',
 'http://www.sothebys.com/en/auctions/lots-archive.html',
 'http://www.sothebys.com/en/auctions/2017/collection-edward-albee-n09678.html',
 'http://www.sothebys.com/en/auctions/2017/contemporary-curated-n09624.html']

In [441]:
len(sothebys_auctions_to_scrape)

9

In [442]:
len(all_sotheby_auctions)

2038

In [445]:
all_sotheby_auctions.extend(all_sotheby_auctions_old)

In [448]:
len(all_sotheby_auctions)

3836

In [449]:
s = list(set(all_sotheby_auctions))

In [450]:
len(s)

1759

In [451]:
thefile = open('sothebys_all_auction_urls.txt', 'w')
for item in s:
  thefile.write("%s\n" % item)


In [452]:
fname = "/Users/denizthemenace/dev/artcollect/data/sothebys_all_auction_urls.txt"
with open(fname) as f:
    content = f.readlines()
#  you may also want to remove whitespace characters like `\n` at the end of each line
urls = [x.strip() for x in content] 

In [455]:
urls

['http://www.sothebys.com/en/auctions/2015/modern-contemporary-south-asian-art-n09479.html',
 'http://www.sothebys.com/en/auctions/2015/fine-classical-chinese-paintings-n09320.html',
 'http://www.sothebys.com/en/auctions/2011/indian-southeast-asian-works-of-art-n08727.html',
 'http://www.sothebys.com/en/auctions/2016/contemporary-art-day-auction-n09573.html',
 'http://www.sothebys.com/en/auctions/2012/important-russian-art.html',
 'http://www.sothebys.com/en/auctions/2010/the-robert-devereux-collection-of-post-war-british-art-in-aid-of-the-african-arts-trust-sale-1-l10143.html',
 'http://www.sothebys.com/en/auctions/2013/arts-dasie-pf1307.html',
 'http://www.sothebys.com/en/auctions/2007/19th-century-european-paintings-am1030.html',
 'http://www.sothebys.com/en/auctions/2007/contemporary-art-l07026.html',
 'http://www.sothebys.com/en/auctions/2015/important-chinese-works-of-art-hk0571.html',
 'http://www.sothebys.com/en/auctions/2012/russian-art-n08844.html',
 'http://www.sothebys.com/

In [458]:
from random import randint
randint(3,6)

3

In [138]:

from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np


# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()


def is_interactive():
    return not hasattr(sys.modules['__main__'], '__file__')

# work-around for Jupyter notebook and IPython console
argv = [] if is_interactive() else sys.argv[1:]
(opts, args) = op.parse_args(argv)
if len(args) > 0:
    op.error("this script takes no arguments.")
    sys.exit(1)


# #############################################################################
# Load some categories from the training set
categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]
# Uncomment the following to do the analysis on all the categories
# categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

dataset = fetch_20newsgroups(subset='all', categories=categories,
                             shuffle=True, random_state=42)

print("%d documents" % len(dataset.data))
print("%d categories" % len(dataset.target_names))
print()

labels = dataset.target
true_k = np.unique(labels).shape[0]

print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
if opts.use_hashing:
    if opts.use_idf:
        # Perform an IDF normalization on the output of HashingVectorizer
        hasher = HashingVectorizer(n_features=opts.n_features,
                                   stop_words='english', alternate_sign=False,
                                   norm=None, binary=False)
        vectorizer = make_pipeline(hasher, TfidfTransformer())
    else:
        vectorizer = HashingVectorizer(n_features=opts.n_features,
                                       stop_words='english',
                                       alternate_sign=False, norm='l2',
                                       binary=False)
else:
    vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                 min_df=2, stop_words='english',
                                 use_idf=opts.use_idf)
X = vectorizer.fit_transform(dataset.data)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()

if opts.n_components:
    print("Performing dimensionality reduction using LSA")
    t0 = time()
    # Vectorizer results are normalized, which makes KMeans behave as
    # spherical k-means for better results. Since LSA/SVD results are
    # not normalized, we have to redo the normalization.
    svd = TruncatedSVD(opts.n_components)
    normalizer = Normalizer(copy=False)
    lsa = make_pipeline(svd, normalizer)

    X = lsa.fit_transform(X)

    print("done in %fs" % (time() - t0))

    explained_variance = svd.explained_variance_ratio_.sum()
    print("Explained variance of the SVD step: {}%".format(
        int(explained_variance * 100)))

    print()


# #############################################################################
# Do the actual clustering

if opts.minibatch:
    km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                         init_size=1000, batch_size=1000, verbose=opts.verbose)
else:
    km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                verbose=opts.verbose)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(labels, km.labels_))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, km.labels_, sample_size=1000))

print()


if not opts.use_hashing:
    print("Top terms per cluster:")

    if opts.n_components:
        original_space_centroids = svd.inverse_transform(km.cluster_centers_)
        order_centroids = original_space_centroids.argsort()[:, ::-1]
    else:
        order_centroids = km.cluster_centers_.argsort()[:, ::-1]

    terms = vectorizer.get_feature_names()
    for i in range(true_k):
        print("Cluster %d:" % i, end='')
        for ind in order_centroids[i, :10]:
            print(' %s' % terms[ind], end='')
        print()


2017-09-12 09:32:18,072 INFO Downloading 20news dataset. This may take a few minutes.
2017-09-12 09:32:18,073 WARNING Downloading dataset from http://people.csail.mit.edu/jrennie/20Newsgroups/20news-bydate.tar.gz (14 MB)


Automatically created module for IPython interactive environment
Usage: __main__.py [options]

Options:
  -h, --help            show this help message and exit
  --lsa=N_COMPONENTS    Preprocess documents with latent semantic analysis.
  --no-minibatch        Use ordinary k-means algorithm (in batch mode).
  --no-idf              Disable Inverse Document Frequency feature weighting.
  --use-hashing         Use a hashing feature vectorizer
  --n-features=N_FEATURES
                        Maximum number of features (dimensions) to extract
                        from text.
  --verbose             Print progress reports inside k-means algorithm.
Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']


2017-09-12 09:32:24,698 INFO Decompressing /Users/denizthemenace/scikit_learn_data/20news_home/20news-bydate.tar.gz


3387 documents
4 categories

Extracting features from the training dataset using a sparse vectorizer
done in 0.962758s
n_samples: 3387, n_features: 10000

Clustering sparse data with MiniBatchKMeans(batch_size=1000, compute_labels=True, init='k-means++',
        init_size=1000, max_iter=100, max_no_improvement=10, n_clusters=4,
        n_init=1, random_state=None, reassignment_ratio=0.01, tol=0.0,
        verbose=False)
done in 0.182s

Homogeneity: 0.583
Completeness: 0.626
V-measure: 0.604
Adjusted Rand-Index: 0.546
Silhouette Coefficient: 0.007

Top terms per cluster:
Cluster 0: graphics university thanks image 3d com ac posting file host
Cluster 1: space access digex pat nasa net shuttle com sci prb
Cluster 2: god com people sandvik don jesus keith article say think
Cluster 3: henry nasa toronto gov alaska space com zoo spencer orbit


In [139]:
import re